### Imports

In [ ]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:

def hex_to_RGB(hex_str):
    """ #FFFFFF -> [255,255,255]"""
    #Pass 16 to the integer function for change of base
    return [int(hex_str[i:i+2], 16) for i in range(1,6,2)]

In [ ]:
def get_color_gradient(c1, c2, n):
    """
    Given two hex colors, returns a color gradient
    with n colors.
    """
    assert n > 1
    c1_rgb = np.array(hex_to_RGB(c1))/255
    c2_rgb = np.array(hex_to_RGB(c2))/255
    mix_pcts = [x/(n-1) for x in range(n)]
    rgb_colors = [((1-mix)*c1_rgb + (mix*c2_rgb)) for mix in mix_pcts]
    return ["#" + "".join([format(int(round(val*255)), "02x") for val in item]) for item in rgb_colors]

In [ ]:
print(*[filename.split('.')[0] for filename in os.listdir('./opinions')], sep='\n')

In [ ]:
product_id = input("Please enter product id")

In [ ]:
opinions = pd.read_json(f'./opinions/{product_id}.json')
opinions

### Calculating basic statistics


In [ ]:
opinions_count = len(opinions)
pros_count = opinions.pros_pl.astype(bool).sum()
cons_count = opinions.cons_pl.astype(bool).sum()
pros_cons_count = opinions.apply(lambda o: bool(o.pros_pl) and bool(o.cons_pl), axis=1).sum()
average_score = opinions.stars.mean()
print(f"Number of opinions about the product: {opinions_count}")
print(f"Number of opinions in which advantages have been listed: {pros_count}")
print(f"Number of opinions in which disadvantages have been listed: {cons_count}")
print(f"Number of opinions in which both advantages and disadvantages have been listed: {cons_count}")
print(f"Average score: {average_score:.2f}")

In [ ]:
pros = opinions.pros_en.explode().value_counts()
cons = opinions.cons_en.explode().value_counts()
print(pros)
print(cons)

In [ ]:
recommendations = opinions.recommend.value_counts(dropna=False).reindex([True, False, np.nan], fill_value=0)
recommendations

In [ ]:
recommendations.plot.pie(
    autopct = lambda r: f"{r:.1f}%" if r>0 else '' ,
    label='',
    colors  = ['forestgreen', 'red', 'blue'],
    labels=['Recommend', 'Not recommend', 'No opinion']
)
plt.title(f"Recommendations for product id: {product_id}\nNumber of opinions: {opinions_count}")
plt.savefig(f"./piecharts/{product_id}.png")
plt.close()

In [ ]:
stars = opinions.stars.value_counts().reindex(list(np.arange(0.5,5.5,0.5)), fill_value=0)
stars

In [82]:
color1='#FF0000'
color2='#0000FF'
plt.figure(figsize=(7,6))
ax = stars.plot.bar(color = get_color_gradient(color1,color2,15))
plt.bar_label(container=ax.containers[0])
plt.xlabel("Number of stars")
plt.ylabel("Number of opinions")
plt.title(f"Number of opinions about product:{product_id}\n with particular")
plt.xticks(rotation=0)
plt.savefig(f"./barcharts/{product_id}.png")
plt.close()